In [1]:
import pandas as pd
import numpy as np
import time
import chess
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def genera_posicion_problema(inicial, moves):
  tablero = chess.Board(inicial)
  movimiento = chess.Move.from_uci(moves.split()[0])
  tablero.push(movimiento)
  problema = tablero.fen()
  return problema

In [3]:
pregunta = input('Descargo el fichero? (Pulsa s para descargarlo): ')

if pregunta.lower() == "s":
  !wget https://database.lichess.org/lichess_db_puzzle.csv.bz2
  !bzip2 -d lichess_db_puzzle.csv.bz2
else:
  pass

Descargo el fichero? (Pulsa s para descargarlo): n


In [4]:
l = pd.read_csv('lichess_db_puzzle.csv', names=['puzzle_id', 'fen', 'moves', 'rating', 'deviation', 'popularity', 'played', 'themes', 'game', 'opening', 'variant'])

In [5]:
l.themes = l.themes.fillna('noTheme')

In [6]:
l.opening = l.opening.fillna('noOpening')

In [7]:
l.variant = l.variant.fillna('noVariant')

In [8]:
inicio = time.perf_counter()

l['problema'] = l.apply(lambda x: genera_posicion_problema(x['fen'], x['moves']), axis=1)
l['sp_san'] = l.apply(lambda x: chess.Board(x['problema']).variation_san([chess.Move.from_uci(m) for m in x['moves'].split()[1:]]), axis=1)

final = time.perf_counter()

duracion = final -inicio

print(f'Proceso terminado en {duracion} segundos.')

Proceso terminado en 666.5165849 segundos.


In [10]:
with open('lichess_puzzles.csv', 'w', encoding="utf-8") as output_csv:
    for index, row in l.iterrows():
        output_csv.write(f"{row['problema']}|{row['opening']}|{row['variant']}|{row['themes']}|{row['puzzle_id']}|{row['game']}|{row['rating']}|´{row['played']}|{row['sp_san']}\n")